In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Activation, RepeatVector, Permute, Multiply, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [ ]:
# Define paths to CSV files
csv_file_paths = [
    'cold_season_NITJ_data.csv',
    'monsoon_season_NITJ_data.csv',
    'post_monsoon_NITJ_data.csv',
    'summer_season_NITJ_data.csv'
]

# Load and combine data from multiple CSV files
dfs = []
for path in csv_file_paths:
    df = pd.read_csv(path, parse_dates=['Date'], index_col='Date')
    dfs.append(df)

data = pd.concat(dfs)

# Define input features (X) and target variable (y)
X = data[['Clearsky GHI', 'Temperature', 'Pressure', 'Relative Humidity',
          'Solar Zenith Angle', 'Wind Speed', 'Wind Direction',
          'Dew Point', 'Precipitable Water', 'Clearness Index']]
y = data['GHI']

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
def build_model(learning_rate, dropout_rate, lstm_units):
    inputs = Input(shape=(X_train.shape[1], 1))

    # Convolutional Layer
    conv = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    pool = MaxPooling1D(pool_size=2)(conv)

    # LSTM Layer
    lstm = LSTM(lstm_units, return_sequences=True)(pool)

    # Attention Mechanism
    attention = Dense(1, activation='tanh')(lstm)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(lstm_units)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = Multiply()([lstm, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(lstm_units,))(sent_representation)

    # Fully Connected Layer
    outputs = Dense(1)(sent_representation)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
# Build the model
model = build_model(learning_rate=0.001, dropout_rate=0.2, lstm_units=64)

# Train the model
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
1405/1405 [==============================] - 9s 5ms/step - loss: 211394.2500 - mae: 375.4690 - val_loss: 182280.8125 - val_mae: 343.6620
Epoch 2/50
1405/1405 [==============================] - 6s 4ms/step - loss: 156666.9062 - mae: 316.3763 - val_loss: 136589.8906 - val_mae: 295.2159
Epoch 3/50
1405/1405 [==============================] - 6s 5ms/step - loss: 118456.7656 - mae: 276.0827 - val_loss: 104933.4219 - val_mae: 263.4168
Epoch 4/50
1405/1405 [==============================] - 5s 4ms/step - loss: 92580.4219 - mae: 248.6922 - val_loss: 82747.0781 - val_mae: 236.8301
Epoch 5/50
1405/1405 [==============================] - 6s 4ms/step - loss: 73809.4609 - mae: 225.0630 - val_loss: 67350.2812 - val_mae: 217.7256
Epoch 6/50
1405/1405 [==============================] - 5s 4ms/step - loss: 50402.3359 - mae: 174.6998 - val_loss: 38654.4453 - val_mae: 142.2658
Epoch 7/50
1405/1405 [==============================] - 6s 4ms/step - loss: 31955.9609 - mae: 128.2541 - val_loss: 270

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

In [ ]:
# Load and preprocess data from file5.csv
file5 = pd.read_csv('cold_season_NITJ_data.csv')
X_file5 = file5.drop(columns=['GHI', 'Date', 'Season'])
y_file5_observed = file5['GHI']

X_file5 = X_file5.astype(np.float32)
X_file5 = np.expand_dims(X_file5, axis=-1)  # Ensure correct input shape

# Use tmodel to predict GHI values for file5.csv
y_file5_predicted = model.predict(X_file5)

# evaluation metrics
rmse = math.sqrt(mean_squared_error(y_file5_observed, y_file5_predicted))
mae = mean_absolute_error(y_file5_observed, y_file5_predicted)
mse = mean_squared_error(y_file5_observed, y_file5_predicted)

# evaluation metrics
print("RMSE:", rmse)
print("MAE:", mae)
print("MSE:", mse)


740/740 [==============================] - 1s 2ms/step
RMSE: 5.737094032928357
MAE: 4.046536737797307
MSE: 32.91424794266216
